In [ ]:
%load_ext autoreload
%autoreload 2


import numpy as np
from utils.data_helper import *
from utils.data import *
from utils.stats import *
from utils.performance import *
from plotly.subplots import make_subplots
from account import Binance
import pandas as pd
import warnings
from strategy_v3.Strategy import *
from strategy_v3.Executor import ExecutorBinance, ExecutorBacktest
from strategy_v3.ExecuteSetup import *
from strategy_v3.ExecuteSetup.StrategyFactory import StrategyFactory
from strategy_v3.DataLoader import DataLoaderBinance
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from time import sleep
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 30)
warnings.filterwarnings('ignore')

In [ ]:
symbol = 'ETHFDUSD'
binance = Binance()
df = binance.get_historical_instrument_price(symbol, interval='15m', start_str='48 hours ago')
plot_price_ohcl(df, symbol)
df['std'] = df['Close'].rolling(15).std()

# Market Making

In [ ]:
symbol = 'BTCFDUSD'
df = Binance().get_historical_instrument_price(symbol, interval='1m', start_str='2 hours ago')
df['std'] = df['Close'].rolling(15).std()
df['adv'] = df['Volume'].rolling(15).mean()
close_px = df['Close'].iloc[-1]
vol = df['std'].iloc[-1]
adv = df['adv'].iloc[-1]

refresh_interval = 30
position_size = 70

spread_flow_factor = 0.3
px_skew_flow_factor = 0.2
gamma = 0.3
target_position = 0
current = 5 * position_size / close_px

limit = 500 if symbol == 'BTCFDUSD' else 200
round = 2 if symbol == 'BTCFDUSD' else 1

df_bid, df_ask = Binance().get_order_book(instrument=symbol, limit=limit)
df_trades_bid, df_trades_ask = Binance().get_aggregate_trades(instrument=symbol, start_date=f'{refresh_interval} seconds ago')
df_trades = pd.concat([df_trades_bid, df_trades_ask])

best_bid = df_bid.iloc[0]['price']
best_ask = df_ask.iloc[0]['price']        
mid_px = (best_ask + best_bid)/2
mkt_sprd = best_ask - best_bid

# MO arrival rate
bid_interval = (df_trades_bid['time'].max() - df_trades_bid['time'].min()).seconds
ask_interval = (df_trades_ask['time'].max() - df_trades_ask['time'].min()).seconds
ar_bid = df_trades_bid['quantity'].sum()/bid_interval
ar_ask = df_trades_ask['quantity'].sum()/ask_interval
ar_skew = ar_ask - ar_bid

# MO arrival to LOB
ar_bid_next = ar_bid * refresh_interval
ar_ask_next = ar_ask * refresh_interval
bid_chg = df_bid[df_bid['quantity_cum'] > abs(ar_bid_next)].iloc[0]['price'] - best_bid
ask_chg = df_ask[df_ask['quantity_cum'] > abs(ar_ask_next)].iloc[0]['price'] - best_ask        
vwmp_skew = (ask_chg + bid_chg) * px_skew_flow_factor
vwmp = mid_px + vwmp_skew

# Target Spread
spread = (mkt_sprd + ask_chg - bid_chg) * spread_flow_factor

r = vwmp - (current - target_position) * gamma * vol ** 2
order_bid = min(r - spread/2, best_bid)
order_ask = max(r + spread/2, best_ask)
spread = order_ask - order_bid

# Skewness
skew_inv = r - vwmp
skew = vwmp - mid_px

print(f'market spread = {mkt_sprd:.4f}, target spread = {spread:.4f}, adv = {adv:.4f}')
print(f'vwmp = {vwmp:.4f}, mid = {mid_px:.4f}, skew = {skew:.4f}, skew_inv = {skew_inv:.4f}')
print(f'arrival bid = {ar_bid:.4f}, arrival ask = {ar_ask:.4f}, arrival_skew = {ar_skew:.4f}')   


fig = make_subplots(rows=2, subplot_titles=['LOB Depth', f'Trades Executed ({refresh_interval} seconds ago)'], vertical_spacing=0.1)
fig.update_layout(title=symbol, width=800, height=1000, hovermode='x')
fig.add_trace(go.Scatter(x=df_bid['price'], y=df_bid['quantity_cum'], name='bid'),row=1, col=1)
fig.add_trace(go.Scatter(x=df_ask['price'], y=df_ask['quantity_cum'], name='ask'),row=1, col=1)

fig.add_vline(order_bid, line_dash="dash", line_color="blue",row=1, col=1)
fig.add_vline(order_ask, line_dash="dash", line_color="red",row=1, col=1)
fig.add_vline(vwmp, line_dash="dash", line_color="green",row=1, col=1, annotation_text='VWMP')
fig.add_vline(r, line_dash="dash", line_color="black",row=1, col=1, annotation_text='R')
fig.update_layout(showlegend=True)

colors = plotly.colors.DEFAULT_PLOTLY_COLORS

fig.add_trace(go.Bar(x=(df_trades_bid['price']/round).round()*round, y=df_trades_bid['quantity'], name='bid', marker={'color': colors[0]}), row=2,col=1)
fig.add_trace(go.Bar(x=(df_trades_ask['price']/round).round()*round, y=df_trades_ask['quantity'], name='ask', marker={'color': colors[1]}), row=2,col=1)
fig.add_vline(mid_px, line_dash="dash", line_color="black",row=2, col=1)

# Strategy performance

In [ ]:
setup = ExecuteSetup.read_all()
strategy_dict = dict()

In [ ]:
for k, v in setup.items():    
    strategy = StrategyFactory().get(k)
    strategy.set_data_loder(DataLoaderBinance())
    strategy.set_executor(ExecutorBinance())
    strategy.set_strategy_id(k)    
    if str(type(strategy)) == str(SimpleMarketMakingStrategy):
        strategy_dict[k] = strategy                           

In [ ]:
# pnl cutoff as 00:00 HKT
offset = 1
date = datetime.today()
date = datetime(year=date.year, month=date.month, day=date.day, tzinfo=ZoneInfo("HongKong")) - timedelta(days=offset)
date_str = date.strftime('%Y-%m-%d %H:%M:%S%z')
date_str_end = (date + timedelta(days=1)).strftime('%Y-%m-%d %H:%M:%S%z')

In [ ]:
for k, v in strategy_dict.items():
    strategy = v
    strategy.load_data(date_str, lookback_end=date_str_end)    
    strategy.summary(True)
    

# Analysis on strategy Log

In [ ]:
strategy = strategy_dict['SMM_BTCv5']
df = strategy.get_log_data()
df = df.sort_values('date')
#df = df.tail(200)
df['px_change'] = df['mid_price'].diff().fillna(0)
df['px_change_next'] = df['px_change'].shift(-1).fillna(0)

In [ ]:
xcol = 'px_change_next'
a1, b1 = np.polyfit(df[xcol].to_numpy(), df['ar_skew'].to_numpy(), 1)
a2, b2 = np.polyfit(df[xcol].to_numpy(), df['skew'].to_numpy(), 1)
a3, b3 = np.polyfit(df[xcol].to_numpy(), df['skew_2'].to_numpy(), 1)

fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=[
        f'Arrival Rate Skew vs {xcol}',     
        f'VWMP Skew vs {xcol}<br>(slope = {a2:.3f})',             
        f'VWMP Skew 2 vs {xcol}<br>(slope = {a3:.3f})',  
    ],     
)
fig.update_layout(
    title=symbol,
    width=1500, height=500,
    showlegend=False,
    xaxis_title=xcol,
    xaxis2_title=xcol,
    xaxis3_title=xcol,
    yaxis_title=f"Arrival Rate Skew",    
    yaxis2_title=f"VWMP Skew",    
    yaxis3_title=f"VWMP Skew",    
)

colors = plotly.colors.DEFAULT_PLOTLY_COLORS
fig.add_trace(go.Scatter(x=df[xcol], y=df['ar_skew'], mode='markers', marker=dict(color=colors[0])), row=1, col=1)
fig.add_trace(go.Scatter(x=df[xcol], y=(a1*df[xcol] + b1), marker=dict(color=colors[0]), mode='lines'), row=1, col=1)             

fig.add_trace(go.Scatter(x=df[xcol], y=df['skew'], mode='markers', marker=dict(color=colors[1])), row=1, col=2)
fig.add_trace(go.Scatter(x=df[xcol], y=(a2*df[xcol] + b2), marker=dict(color=colors[1]), mode='lines'), row=1, col=2)             

fig.add_trace(go.Scatter(x=df[xcol], y=df['skew_2'], mode='markers', marker=dict(color=colors[1])), row=1, col=3)
fig.add_trace(go.Scatter(x=df[xcol], y=(a3*df[xcol] + b3), marker=dict(color=colors[1]), mode='lines'), row=1, col=3)             